# Analysis of mitigation potential of five strategies:

 S1. Decarbonization of on-site electricity consumption (switch to wind power)

 S2. Decarbonization of process heat (electrification of heat)

 S3. Decarbonization of electricity in chemical indsutry (switch to wind power)

 S4. Decarbonization of process heat in chemical industry (electrification of heat)

### Basic set-up

In [1]:
%run _imports.ipynb

In [2]:
BW_PROJECT = 'lib_rm' # insert your project name here
bw.projects.set_current(BW_PROJECT)

EI_DB_NAME = 'ecoinvent-3.10-cutoff' # name of ecoinvent database in your project
LIB_RM_DB_NAME = "EV battery metals"

INVENTORIES = {
    "Lithium, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN")
}

IMPACT_METHODS = {"Climate change": ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)')}

technosphere = lambda x: x["type"] == "technosphere"
biosphere = lambda x: x["type"] == "biosphere"

breakdown_lists = supporting_functions.get_breakdown_lists()
inventories_breakdown = pd.read_excel(INVENTORIES_PATH, sheet_name="Datasets for breakdown", index_col=0, skiprows=1)

In [25]:
dbs_strategies = {
    "BAU": LIB_RM_DB_NAME,
    "ONSITE_ELEC": LIB_RM_DB_NAME + ", wind power on-site",
    "PROCESS_HEAT": LIB_RM_DB_NAME + ", + electrification process heat",
    "CHEMICALS_ELEC": LIB_RM_DB_NAME + ", + wind power in chemical industry",
    "CHEMICALS_HEAT": LIB_RM_DB_NAME + ", + electrification heat in chemical industry",
    "GLOBAL_ELEC": LIB_RM_DB_NAME + ", + wind power in all activities",
}

# Datasets used to model decarbonization strategies

# On-site electricity source shifts to wind onshore
wind_filter = {
    'name': "electricity production, wind, 1-3MW turbine, onshore",
    'product': "electricity, high voltage",
    'unit': "kilowatt hour"
    }

# Process heat is electrified
heat_low_temp_filter = {
    'name': "heat production, at heat pump 30kW, allocation exergy",
    'product': "heat, central or small-scale, other than natural gas",
    'unit': "megajoule"
    }

heat_high_temp_filter = {
    'name': "heat production, electric furnace",
    'product': "heat, district or industrial, other than natural gas",
    'unit': "megajoule"
    }

In [4]:
# Import ecoinvent database into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(EI_DB_NAME)

# Import LIB raw materials inventories into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(lib_rm_db)
except NameError:
  lib_rm_db = wurst.extract_brightway2_databases(LIB_RM_DB_NAME)

biosphere_db = [ef.as_dict() for ef in bw.Database('ecoinvent-3.10-biosphere')]

Getting activity data


100%|██████████| 23523/23523 [00:00<00:00, 94255.15it/s]


Adding exchange data to activities


100%|██████████| 743409/743409 [00:59<00:00, 12526.99it/s]


Filling out exchange data


...

100%|██████████| 23/23 [00:00<00:00, 46290.30it/s]


Adding exchange data to activities


100%|██████████| 447/447 [00:00<00:00, 29384.12it/s]


Filling out exchange data


100%|██████████| 23/23 [00:00<00:00, 131.56it/s]


### 1. Decarbonization of on-site electricity consumption

In [5]:
# Copy of databases for modification:
lib_rm_db_s1 = copy.deepcopy(lib_rm_db)

In [6]:
# Get the list of possible wind power datasets
possible_wind_datasets = list(wurst.transformations.geo.get_possibles(wind_filter, ei_db)) 

for ds in lib_rm_db_s1:
    if ds["name"] in list(inventories_breakdown["name"]):
        
        # Check if there is an exact wind dataset match for the location
        wind_match_dataset = [a for a in possible_wind_datasets if a['location'] == ds["location"]]

        # If there is no specific dataset for the location, use RoW
        if len(wind_match_dataset) == 0:
            wind_match_dataset = [a for a in possible_wind_datasets if a['location'] == 'RoW']

        # Replace electricity grid with wind power in the activities
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:
                exc.update({
                    'name': wind_match_dataset[0]['name'],
                    'product': wind_match_dataset[0]['reference product'],
                    'location': wind_match_dataset[0]['location'],
                    'input': (wind_match_dataset[0]['database'], wind_match_dataset[0]['code'])})

In [8]:
if dbs_strategies["ONSITE_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["ONSITE_ELEC"]]
wurst.write_brightway2_database(lib_rm_db_s1, dbs_strategies["ONSITE_ELEC"])

Writing activities to SQLite3 database:


23 datasets
447 exchanges
0 unlinked exchanges
  


0% [################       ] 100% | ETA: 00:00:00

0% [#######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/01/2024 21:07:53
  Finished: 04/01/2024 21:07:54
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 36.72
Created database: EV battery metals, wind power on-site


### 2. + Electrification of process heat

In [7]:
# Copy of databases for modification:
lib_rm_db_s2 = copy.deepcopy(lib_rm_db_s1)

In [8]:
# Create regionalized heat pump and electric furnace datasets:
list_countries = list(set([ds["location"] for ds in lib_rm_db_s2 if ds["location"] not in ["GLO", "RoW"]])) + ["RoW"]

# Get heat pump and electric furnace dataset
heat_pump_ds = supporting_functions.get_dataset_for_location(heat_low_temp_filter, "RoW", ei_db)
elec_furnace_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, "GLO", lib_rm_db)

for c in list_countries:
    # Get wind power
    wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, c, ei_db)

    # Create heat pump and electric furnace datasets for location
    heat_pump_loc = supporting_functions.replicate_activity_to_location(heat_pump_ds, c, ei_db, dbs_strategies["PROCESS_HEAT"])
    elec_furnace_loc = supporting_functions.replicate_activity_to_location(elec_furnace_ds, c, ei_db, dbs_strategies["PROCESS_HEAT"])

    #... and substitute electricity source by wind
    for heat_ds in [heat_pump_loc, elec_furnace_loc]:
        for exc in filter(technosphere, heat_ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:
                exc.update({
                        'name': wind_ds_loc['name'],
                        'product': wind_ds_loc['reference product'],
                        'location': wind_ds_loc['location'],
                        'input': (wind_ds_loc['database'], wind_ds_loc['code'])})
    
        lib_rm_db_s2.append(heat_ds)

In [9]:
# Substitute heat sources in raw material datasets:

low_temperature_ds = [
    "lithium brine production, from evaporation pond, Salar de Atacama",
    "lithium brine purification, Salar de Atacama",
    "lithium carbonate production, Salar de Atacama",
    "lithium hydroxide production, Salar de Atacama"
]

for ds in lib_rm_db_s2:
    if ds["name"] in list(inventories_breakdown["name"]):

        # Low-temperature heat is supplied by heat pumps
        if ds["name"] in low_temperature_ds:
            electrified_heat_ds = supporting_functions.get_dataset_for_location(heat_low_temp_filter, ds["location"], lib_rm_db_s2)
        else:
            # Nickel sulfate is a global activity
            if ds["location"] == "GLO":
                electrified_heat_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, "RoW", lib_rm_db_s2)
            else:
                electrified_heat_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, ds["location"], lib_rm_db_s2)

        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in breakdown_lists["heating products"]:
                exc.update({
                    'name': electrified_heat_ds['name'],
                    'product': electrified_heat_ds['reference product'],
                    'location': electrified_heat_ds['location'],
                    "database": electrified_heat_ds['database'],
                    'input': (electrified_heat_ds['database'], electrified_heat_ds['code'])})

In [12]:
if dbs_strategies["PROCESS_HEAT"] in bw.databases:
    del bw.databases[dbs_strategies["PROCESS_HEAT"]]
wurst.write_brightway2_database(lib_rm_db_s2, dbs_strategies["PROCESS_HEAT"])

Writing activities to SQLite3 database:


33 datasets
477 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/02/2024 11:32:31
  Finished: 04/02/2024 11:32:31
  Total time elapsed: 00:00:00
  CPU %: 60.10
  Memory %: 36.28
Created database: EV battery metals, + electrification process heat


### 3.  + Decarbonization of electricity in reagents production

In [10]:
# Copy of databases for modification:
lib_rm_db_s3 = copy.deepcopy(lib_rm_db_s2)
ei_db_s3 = copy.deepcopy(ei_db)

EI_CHEMICALS_WIND = EI_DB_NAME + ", reagents production powered by wind"

In [11]:
# Since some of the chemicals used in LIB raw materials rely on
# other chemicals produced upstream, we need to ensure that
# all upstreams processes also switch to wind power. Therefore,
# we change all the datasets falling under these categories:

chemicals_ISIC_categories = [
    '1920:Manufacture of refined petroleum products',
    '1040:Manufacture of vegetable and animal oils and fats',
    '2394:Manufacture of cement, lime and plaster',
    '0891:Mining of chemical and fertilizer minerals',
    '2011:Manufacture of basic chemicals',
    '0810:Quarrying of stone, sand and clay',
    '2012:Manufacture of fertilizers and nitrogen compounds',
    '2030:Manufacture of man-made fibres',
    '2013:Manufacture of plastics and synthetic rubber in primary forms',
    '2100:Manufacture of pharmaceuticals, medicinal chemical and botanical products',
    '2029:Manufacture of other chemical products n.e.c.',
    '2220:Manufacture of plastics products',
    '201:Manufacture of basic chemicals, fertilizers and nitrogen compounds, plastics',
    '20:Manufacture of chemicals and chemical products',
    '2023:Manufacture of soap and detergents, cleaning and polishing preparations, pe'
]

In [12]:
# Create an ecoinvent database where reagents production is powered by wind
for ds in ei_db_s3:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']
    
    if len(product_category) > 0:
        if product_category[0] in chemicals_ISIC_categories:

            for exc in filter(technosphere, ds["exchanges"]):
                if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:

                    # Get wind power
                    try:
                        wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, exc["location"], ei_db_s3)
                        if wind_elec_loc["location"] != exc["location"]:
                            wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db_s3)
                    except KeyError:
                        wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db_s3)
                    
                    exc.update({
                            'name': wind_elec_loc['name'],
                            'product': wind_elec_loc['reference product'],
                            'location': wind_elec_loc['location'],
                            'input': (wind_elec_loc['database'], wind_elec_loc['code'])}
                        )

BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex


In [13]:
# Write the modified ecoinvent database to memory
if EI_CHEMICALS_WIND in bw.databases:
    del bw.databases[EI_CHEMICALS_WIND]
wurst.write_brightway2_database(ei_db_s3, EI_CHEMICALS_WIND)

Vacuuming database 
23523 datasets
743409 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:55


Title: Writing activities to SQLite3 database:
  Started: 04/02/2024 19:05:24
  Finished: 04/02/2024 19:06:20
  Total time elapsed: 00:00:55
  CPU %: 91.50
  Memory %: 28.00
Created database: ecoinvent-3.10-cutoff, reagents production powered by wind


In [14]:
# Link raw materials datasets to the modified ecoinvent version
for ds in lib_rm_db_s3:
    for exc in filter(technosphere, ds["exchanges"]):
        if EI_DB_NAME in exc["database"]:
            exc_ds = [
                d for d in bw.Database(EI_CHEMICALS_WIND) if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]
                
            exc.update(
                {
                    "input": (exc_ds["database"], exc_ds["code"])
                }
            )

In [15]:
if dbs_strategies["CHEMICALS_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["CHEMICALS_ELEC"]]
wurst.write_brightway2_database(lib_rm_db_s3, dbs_strategies["CHEMICALS_ELEC"])

Writing activities to SQLite3 database:


33 datasets
477 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/02/2024 19:14:17
  Finished: 04/02/2024 19:14:17
  Total time elapsed: 00:00:00
  CPU %: 40.10
  Memory %: 29.03
Created database: EV battery metals, + wind power in chemical industry


### 4. + Electrification of heat in chemical industry

In [16]:
# Copy of databases for modification:
lib_rm_db_s4 = copy.deepcopy(lib_rm_db_s3)
ei_db_s4 = copy.deepcopy(ei_db_s3)

EI_CHEMICALS_WIND_AND_HEAT = EI_DB_NAME + ", reagents production powered by wind + electrified heat"

In [17]:
# Copy electric furnace dataset to the modified ecoinvent database
elec_furnace_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, "GLO", lib_rm_db)
elec_furnace_ei = supporting_functions.replicate_activity_to_location(elec_furnace_ds, "GLO", ei_db, EI_CHEMICALS_WIND_AND_HEAT)

# Cange electricity to wind
wind_ds_loc = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db_s4)

# Change electricity source to wind
for exc in filter(technosphere, elec_furnace_ei["exchanges"]):
    if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:
        exc.update({
            'name': wind_ds_loc['name'],
            'product': wind_ds_loc['reference product'],
            'location': wind_ds_loc['location'],
            'input': (wind_ds_loc['database'], wind_ds_loc['code'])}
            )

# Add electric furnace dataset to ecoinvent
ei_db_s4.append(elec_furnace_ei)

In [18]:
# Create an ecoinvent database where reagents production is powered by wind
for ds in ei_db_s4:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']
    
    if len(product_category) > 0:
        if product_category[0] in chemicals_ISIC_categories:

            for exc in filter(technosphere, ds["exchanges"]):
                if exc['product'] in breakdown_lists["heating products"]:
                    exc.update({
                        'name': elec_furnace_ei['name'],
                        'product': elec_furnace_ei['reference product'],
                        'location': elec_furnace_ei['location'],
                        "database": elec_furnace_ei['database'],
                        'input': (elec_furnace_ei['database'], elec_furnace_ei['code'])}
                    )

In [19]:
# Write the modified ecoinvent database to memory
if EI_CHEMICALS_WIND_AND_HEAT in bw.databases:
    del bw.databases[EI_CHEMICALS_WIND_AND_HEAT]
wurst.write_brightway2_database(ei_db_s4, EI_CHEMICALS_WIND_AND_HEAT)

Vacuuming database 
23524 datasets
743411 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Title: Writing activities to SQLite3 database:
  Started: 04/02/2024 19:16:14
  Finished: 04/02/2024 19:16:56
  Total time elapsed: 00:00:42
  CPU %: 99.70
  Memory %: 30.92
Created database: ecoinvent-3.10-cutoff, reagents production powered by wind + electrified heat


In [20]:
# Link raw materials datasets to the modified ecoinvent version
for ds in lib_rm_db_s4:
    for exc in filter(technosphere, ds["exchanges"]):
        if EI_DB_NAME in exc["database"]:
            exc_ds = [
                d for d in bw.Database(EI_CHEMICALS_WIND_AND_HEAT) if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]
                
            exc.update(
                {
                    "input": (exc_ds["database"], exc_ds["code"])
                }
            )

In [21]:
if dbs_strategies["CHEMICALS_HEAT"] in bw.databases:
    del bw.databases[dbs_strategies["CHEMICALS_HEAT"]]
wurst.write_brightway2_database(lib_rm_db_s4, dbs_strategies["CHEMICALS_HEAT"])

Writing activities to SQLite3 database:


33 datasets
477 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/02/2024 19:25:36
  Finished: 04/02/2024 19:25:36
  Total time elapsed: 00:00:00
  CPU %: 45.70
  Memory %: 31.58
Created database: EV battery metals, + electrification heat in chemical industry


#### 5. + Wind power in the global economy

In [26]:
# Copy of databases for modification:
lib_rm_db_s5 = copy.deepcopy(lib_rm_db_s4)
ei_db_s5 = copy.deepcopy(ei_db_s4)

EI_WIND_GLOBAL = EI_DB_NAME + ", reagents production powered by wind + electrified heat + global wind"

In [27]:
# Create an ecoinvent database where all activities are powered by wind
for ds in ei_db_s5:
    for exc in filter(technosphere, ds["exchanges"]):
        if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:

            # Get wind power
            try:
                wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, exc["location"], ei_db_s5)
                if wind_elec_loc["location"] != exc["location"]:
                    wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db_s5)
            except KeyError:
                wind_elec_loc = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db_s5)
                    
            exc.update({
                'name': wind_elec_loc['name'],
                'product': wind_elec_loc['reference product'],
                'location': wind_elec_loc['location'],
                'input': (wind_elec_loc['database'], wind_elec_loc['code'])}
            )

CN-ECGC not found in regex
BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NCGC not found in regex
CN-CCG not found in regex
CN-NWG not found in regex
CN-NECG not found in regex
CN-NWG not found in regex
CN-NWG not found in regex
CN-NWG not found in regex
CN-CCG not found in regex
CN-NWG not found in regex
CN-SWG not found in regex
CN-NCGC not found in regex
CN-NECG not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NWG not found in regex
CN-NWG not found in regex
CN-ECGC not found in regex
CN-NWG not found in regex
CN-NECG not found in regex
CN-CCG not found in regex
CN-NCGC not found in regex
CN-SWG not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NECG not found in regex
CN-ECGC not found in regex
CN-NCGC not found in regex
CN-CCG not found in regex
CN-NWG not found in regex
CN-SWG not found in regex
CN-NCGC not found in r

In [28]:
# Write the modified ecoinvent database to memory
if EI_WIND_GLOBAL in bw.databases:
    del bw.databases[EI_WIND_GLOBAL]
wurst.write_brightway2_database(ei_db_s5, EI_WIND_GLOBAL)

23524 datasets
743411 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:41


Title: Writing activities to SQLite3 database:
  Started: 04/04/2024 13:14:38
  Finished: 04/04/2024 13:16:20
  Total time elapsed: 00:01:41
  CPU %: 80.20
  Memory %: 20.82
Created database: ecoinvent-3.10-cutoff, reagents production powered by wind + electrified heat + global wind


In [29]:
# Link raw materials datasets to the modified ecoinvent version
for ds in lib_rm_db_s5:
    for exc in filter(technosphere, ds["exchanges"]):
        if EI_DB_NAME in exc["database"]:
            exc_ds = [
                d for d in bw.Database(EI_WIND_GLOBAL) if 
                d["name"] == exc["name"] and
                d["reference product"] == exc["product"] and
                d["location"] == exc["location"] and
                d["unit"] == exc["unit"]
                ][0]
                
            exc.update(
                {
                    "input": (exc_ds["database"], exc_ds["code"])
                }
            )

In [34]:
if dbs_strategies["GLOBAL_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["GLOBAL_ELEC"]]
wurst.write_brightway2_database(lib_rm_db_s5, dbs_strategies["GLOBAL_ELEC"])

Writing activities to SQLite3 database:


33 datasets
477 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/04/2024 13:35:35
  Finished: 04/04/2024 13:35:35
  Total time elapsed: 00:00:00
  CPU %: 40.10
  Memory %: 4.53
Created database: EV battery metals, + wind power in all activities


### Calculte mitigation potential 

In [35]:
mitigation_potential = {
        db: {
            rm: 0 
            for rm in INVENTORIES.keys()} 
            for db in list(dbs_strategies.keys())
        }

for db in dbs_strategies:
    for rm in INVENTORIES:
        rm_ds = [ds for ds in bw.Database(dbs_strategies[db])
                    if ds['name'] == INVENTORIES[rm][0] 
                    and ds['reference product'] == INVENTORIES[rm][1]
                    and ds['location'] == INVENTORIES[rm][2]][0]
        impacts = supporting_functions.multi_lcia(rm_ds, IMPACT_METHODS)
        mitigation_potential[db][rm] = impacts

In [36]:
mitigation_potential = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in mitigation_potential.items()
}

In [37]:
# Export mitigation potential results
pd.DataFrame(mitigation_potential).to_csv(DATA_DIR / "results" / f"fig4_actual_mitigaiton_potential_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")